In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
import pandas as pd
import numpy as np
from typing import Union, List, Tuple, Dict, Any, Optional

from skforecast.utils import preprocess_y
from skforecast.utils import preprocess_exog

In [6]:
y = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y.index = pd.date_range(start='2020-01-01', periods=len(y), freq='D')
display(y)

exog = pd.Series([4, 5, 6, 7])
exog.index = pd.date_range(start='2020-01-04', periods=len(exog), freq='D')
exog = exog.iloc[[0, 2, 3]]
display(exog)

2020-01-01     1
2020-01-02     2
2020-01-03     3
2020-01-04     4
2020-01-05     5
2020-01-06     6
2020-01-07     7
2020-01-08     8
2020-01-09     9
2020-01-10    10
Freq: D, dtype: int64

2020-01-04    4
2020-01-06    6
2020-01-07    7
dtype: int64

In [7]:
index_intersection = y.index.intersection(exog.index)
index_intersection

DatetimeIndex(['2020-01-04', '2020-01-06', '2020-01-07'], dtype='datetime64[ns]', freq=None)

In [11]:
type(None)

NoneType

In [9]:
lags = np.arange(200)

In [14]:
if True and [1, 2, 3] == None:
    print(1)

In [12]:
import pandas as pd

type(pd.DataFrame()).__name__

'DataFrame'

In [13]:
input_series_is_dict = isinstance(y, dict)

'dict'

In [10]:
%%timeit -n 1000

x = [f"lag_{i}" for i in lags]

48.8 µs ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
if len(index_intersection) != len(y):
    exog_level = exog.reindex(y.index, fill_value=np.nan)

exog_level

2020-01-01    NaN
2020-01-02    NaN
2020-01-03    NaN
2020-01-04    4.0
2020-01-05    NaN
2020-01-06    6.0
2020-01-07    7.0
2020-01-08    NaN
2020-01-09    NaN
2020-01-10    NaN
Freq: D, dtype: float64

In [49]:
def check_series(
    series = Union[pd.DataFrame, Dict[str, pd.Series], Dict[str, pd.DataFrame]]
)-> None:
    """
    """

    if not isinstance(series, pd.DataFrame, dict):
        raise TypeError(
            (
                f"`series` must be a pandas DataFrame or a dict of DataFrames or Series. "
                f"Got {type(series)}."
            )
        )

    if isinstance(series, dict):
        not_valid_series = [
            k
            for k, v in series.items()
            if not isinstance(v, (pd.Series, pd.DataFrame))
        ]
        if not_valid_series:
            raise TypeError(
                (
                    f"All series must be a named pandas Series or a pandas Dataframe. "
                    f"with a single column. Review series: {not_valid_series}"
                )
            )

        for k, v in series.items():
            if isinstance(v, pd.DataFrame):
                if v.shape[1] != 1:
                    raise ValueError(
                        (
                            f"All series must be a named pandas Series or a pandas Dataframe "
                            f"with a single column. Review series: {k}"
                        )
                    )

        not_valid_index = [
            k
            for k, v in series.items()
            if not isinstance(v.index, pd.DatetimeIndex)
        ]
        if not_valid_index:
            raise TypeError(
                (
                    f"All series must have a DatetimeIndex index with the same frequency. "
                    f"Review series: {not_valid_index}"
                )
            )

        indexes_freq = [v.index.freq for v in series.values()]
        if not len(set(indexes_freq)) == 1:
            raise ValueError(
                (
                    f"All series must have a DatetimeIndex index with the same frequency. "
                    f"Found frequencies: {set(indexes_freq)}"
                )
            )


In [ ]:
def check_exog_multiseries(
    series: Union[pd.DataFrame, Dict[str, pd.Series], Dict[str, pd.DataFrame]],
    exog: Optional[
        Union[pd.DataFrame, pd.Series, Dict[str, pd.Series], Dict[str, pd.DataFrame]]
    ] = None,
) -> None:

    if exog is None:
        return

    if not isinstance(exog, (pd.Series, pd.DataFrame, dict)):
        raise TypeError(
            (
                f"`exog` must be a pandas Series, DataFrame or dict. "
                f"Got {type(exog)}."
            )
        )

    if isinstance(exog, (pd.Series, pd.DataFrame)):
        if isinstance(series, dict):
            raise TypeError(
                (
                    f"`exog` must be a dict of DataFrames or Series if "
                    f"`series` is a dict. Got {type(exog)}."
                )
            )

        # TODO: disable warnings of this two wxecutions
        _, y_index = preprocess_y(y=series, return_values=False)
        _, exog_index = preprocess_exog(exog=exog, return_values=False)
        if not (exog_index == series_index).all():
            raise ValueError(
                (
                    "Different index for `series` and `exog`. They must be equal "
                    "to ensure the correct alignment of values."
                )
            )
    else:
        # Only elements already present in exog_dict are updated
        series_names = set(series.keys()) if isinstance(series, dict) else set(series.columns)
        series_not_in_exog = set(series_names) - set(exog.keys())
        if series_not_in_exog:
            warnings.warn(
                (
                    f"{series_not_in_exog} not present in `exog`. All values "
                    f"of the exogenous variables for these series will be NaN."
                ),
                MissingValuesExogWarning,
            )

        for k, v in exog.items():
            check_exog(exog=v, allow_nan=True)

        if isinstance(series, pd.DataFrame):
            for k, v in exog_dict.items():
                if v is not None:
                    _, v_index = preprocess_exog(exog=v, return_values=False)
                    exog_dict[k].index = v_index
                    if (v.index == series.index).all():
                        raise ValueError(
                            (
                                f"Different index for `series` {k} and its `exog`. "
                                f"When `series` is a pandas DataFrame, they must "
                                f"be equal to ensure the correct alignment of values."
                            )
                        )
        else:
            not_valid_index = [
                k
                for k, v in exog_dict.items()
                if v is not None and not isinstance(v.index, pd.DatetimeIndex)
            ]
            if not_valid_index:
                raise TypeError(
                    (
                        f"All exog must have a DatetimeIndex index with the same frequency. "
                        f"Review exog: {not_valid_index}"
                    )
                )

            indexes_freq = [
                v.index.freq for k, v in exog_dict.items() if v is not None
            ]
            if not len(set(indexes_freq)) == 1:
                raise ValueError(
                    (
                        f"All exog must have a DatetimeIndex index with the same frequency. "
                        f"Found frequencies: {set(indexes_freq)}"
                    )
                )

    exog_col_names = list(
        set(column for df in exog_dict.values() for column in df.columns.to_list())
    )

    # Check that all exog have the same dtypes for common columns
    exog_dtype_dict = {col_name: set() for col_name in exog_col_names}
    for v in exog_dict.values():
        if v is not None:
            for col_name in v.columns:
                exog_dtype_dict[col_name].add(v[col_name].dtype.name)

    for col_name, dtypes in exog_dtype_dict.items():
        if len(dtypes) > 1:
            raise TypeError(
                (
                    f"Column {col_name} has different dtypes in different exog "
                    f"DataFrames or Series."
                )
            )

    if len(set(exog_col_names) - set(series_col_names)) != len(exog_col_names):
        raise ValueError(
            (
                f"`exog` cannot contain a column named the same as one of the "
                f"series (column names of series).\n"
                f"    `series` columns : {series_col_names}.\n"
                f"    `exog`   columns : {exog_col_names}."
            )
        )


In [ ]:
def check_input_multiseries(
    series: Union[pd.DataFrame, Dict[str, pd.Series], Dict[str, pd.DataFrame]],
    exog: Optional[
        Union[pd.DataFrame, pd.Series, Dict[str, pd.Series], Dict[str, pd.DataFrame]]
    ] = None,
) -> None:
    """
    Check input data for ForecasterMultiseries.

    Parameters
    ----------
    series : pandas.DataFrame or dict of pandas.Series or dict of pandas.DataFrame
        Input time series data.
    exog : pandas.DataFrame or dict of pandas.Series or dict of pandas.DataFrame, default None
        Exogenous variables data.

    Returns
    -------
    None
    """

    



def preprocess_multiseries_input(
    series: Union[pd.DataFrame, Dict[str, pd.Series], Dict[str, pd.DataFrame]],
    exog: Optional[Union[pd.DataFrame, pd.Series, Dict[str, pd.Series], Dict[str, pd.DataFrame]]] = None
) -> Tuple[Dict[str, pd.Series], Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]:

    if isinstance(series, pd.DataFrame):
        _, series_index = preprocess_y(y=series, return_values=False)
        series_dict = series.copy()
        series_dict.index = series_index
        series_dict = series.to_dict("series")

    # Remove leading and trailing nans from each series.
    for k, v in series_dict.items():
        series_dict[k] = v.loc[v.first_valid_index() : v.last_valid_index()]


In [50]:
series = pd.DataFrame(
    data=np.random.rand(4, 3),
    index=pd.date_range(start="2020-01-01", periods=4, freq="D"),
    columns=["y_1", "y_2", "y_3"]
)

check_and_preprocess_input_multiseries(series=series)

In [53]:
series = {
    "series_1": pd.DataFrame(
        data=np.random.rand(4, 1),
        index=pd.date_range(start="2020-01-01", periods=4, freq="D"),
        columns=["y_1"]
    ),
    "series_2": pd.DataFrame(
        data=np.random.rand(4, 1),
        index=pd.date_range(start="2020-01-01", periods=4, freq="D"),
        columns=["y_2"]
    ),
}

check_and_preprocess_input_multiseries(series=series)

In [51]:
series = pd.Series(
    data=np.random.rand(4),
    index=pd.date_range(start="2020-01-01", periods=4, freq="D"),
    name="y"
)

check_and_preprocess_input_multiseries(series=series)

TypeError: `series` must be a pandas DataFrame or a dict of DataFrames or Series. Got <class 'pandas.core.series.Series'>.

In [34]:
series = pd.DataFrame(
    data=np.random.rand(4, 3),
    index=pd.date_range(start="2020-01-01", periods=4, freq="D"),
    columns=["y_1", "y_2", "y_3"]
)
exog = pd.Series(
    data=[1, 2, 3],
    index=pd.date_range(start="2020-01-01", periods=3, freq="D"),
    name="exog_1"
)

display(series.head())
display(exog.head())

check_input_multiseries(series=series, exog=exog)

,y_1,y_2,y_3
2020-01-01,0.040225,0.941863,0.266589
2020-01-02,0.828271,0.139975,0.962412
2020-01-03,0.895854,0.329336,0.309633
2020-01-04,0.673742,0.326813,0.976502


2020-01-01    1
2020-01-02    2
2020-01-03    3
Freq: D, Name: exog_1, dtype: int64